# **Convolutional Neural Network for Image Classification**
*   Implementation of a convolutional neural network classifier supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a convolutional neural network classifier supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementations are based off of TensorFlow's very own example: [TensorFlow CNN Tutorial](https://www.tensorflow.org/tutorials/images/cnn)

In [1]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

In [2]:
# Set a global random seed
tf.random.set_seed(12)

In [3]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 3
# Values are specific to the Fashion MNIST dataset
INPUT_DIM = 28
INPUT_CHANNELS = 1
NUM_CLASSES = 10

In [4]:
def build_and_train(X_train, y_train, precision='single'):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS), dtype=dtype),
        layers.MaxPooling2D((2, 2), dtype=dtype), 
        layers.Conv2D(64, (3, 3), activation='relu', dtype=dtype),
        layers.MaxPooling2D((2, 2), dtype=dtype), 
        layers.Conv2D(64, (3, 3), activation='relu', dtype=dtype),
        layers.Flatten(dtype=dtype),
        layers.Dense(64, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, activation='softmax', dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [5]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(INPUT_DIM, INPUT_DIM, INPUT_CHANNELS)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [6]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test = X_test / 255.0

4422102/4422102 [==============================] - 2s 0us/step


In [ ]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.4861 - accuracy: 0.8208
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3190 - accuracy: 0.8833
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2726 - accuracy: 0.8996
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2442 - accuracy: 0.9104
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2204 - accuracy: 0.9183
Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.4938 - accuracy: 0.8180
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3219 - accuracy: 0.8816
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2752 - accuracy: 0.8985
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2458 - accuracy: 0.9089
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2210 - accura

In [10]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 17s 7ms/step - loss: 0.4865 - accuracy: 0.8192
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3149 - accuracy: 0.8853
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2688 - accuracy: 0.9011
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2391 - accuracy: 0.9113
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2178 - accuracy: 0.9183
Epoch 1/5
1875/1875 [==============================] - 16s 7ms/step - loss: 0.4911 - accuracy: 0.8181
Epoch 2/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.3190 - accuracy: 0.8831
Epoch 3/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2729 - accuracy: 0.8996
Epoch 4/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2446 - accuracy: 0.9089
Epoch 5/5
1875/1875 [==============================] - 13s 7ms/step - loss: 0.2215

In [20]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.4866 - accuracy: 0.8229
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3193 - accuracy: 0.8834
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2741 - accuracy: 0.8995
Epoch 4/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2471 - accuracy: 0.9086
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2232 - accuracy: 0.9175
Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.5093 - accuracy: 0.8107
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3329 - accuracy: 0.8781
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2849 - accuracy: 0.8956
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2550 - accuracy: 0.9053
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2306 - accurac

In [16]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 2/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 3/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 4/5
1875/1875 [==============================] - 7s 4ms/step - loss: nan - accuracy: 0.1000
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: nan - accuracy: 0.1000
Epoch 1/5
1875/1875 

In [22]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.5040 - accuracy: 0.8172
Epoch 2/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3262 - accuracy: 0.8808
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2812 - accuracy: 0.8975
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2520 - accuracy: 0.9071
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2307 - accuracy: 0.9144
Epoch 1/5
1875/1875 [==============================] - 11s 5ms/step - loss: 0.5142 - accuracy: 0.8111
Epoch 2/5
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3352 - accuracy: 0.8771
Epoch 3/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2854 - accuracy: 0.8951
Epoch 4/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2532 - accuracy: 0.9062
Epoch 5/5
1875/1875 [==============================] - 9s 5ms/step - loss: 0.2296 - accur

In [23]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 73.5109293460846 seconds
Average training time in single precision: 39.506595293680824 seconds
Average training time in half precision: 40.641441424687706 seconds
Average training time in mixed half precision: 59.07044339179993 seconds
-------------
Accuracy with double precision: 0.9017000198364258
Accuracy with single precision: 0.9025999903678894
Accuracy with half precision: 0.10000000149011612
Accuracy with mixed half precision: 0.9000999927520752
